In [15]:
%pip install import-ipynb

   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   --------------- ------------------------ 30.7/78.5 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 78.5/78.5 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
from scipy.spatial import distance_matrix
import re
import import_ipynb
import numpy as np
import solver

importing Jupyter notebook from solver.ipynb
Note: you may need to restart the kernel to use updated packages.


In [23]:
def load_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Dictionary of values
    values = {}
    
    # Lists to save coordinates and weights
    coordinates = []
    weights = []

    for line in lines:
        if ',' in line:
            key, value = line.strip().split(',')
            if key in ["NUM DRONES", "NUM TRUCKS"]:
                values[key] = int(value)
            elif key in ["TRUCK CAP", "DRONE CAP", "TRUCK SPEED", "DRONE SPEED", "DRONE ENDURANCE", "DRONE TIME LIMIT", "TRUCK TIME LIMIT", "TRUCK UNIT COST", "DRONE UNIT COST"]:
                values[key] = float(value)
        else:
            parts = re.sub(' +',' ', line).split()[1:] 
            # Add coordinates and weights to the lists
            x, y, weight = map(float, parts)
            coordinates.append((x, y))
            weights.append(weight)

    return values, coordinates, weights

# Load instance
file_path = "./instances/test_instance copy.txt"  # Instance path
values, coordinates, weights = load_instance(file_path)

# Print values, coordinates and weights
print("Values:", values)

print("Coordinate:")
for coord in coordinates:
    print(coord)

print("Pesi:")
for weight in weights:
    print(weight)

Values: {'NUM DRONES': 4, 'NUM TRUCKS': 10, 'TRUCK CAP': 1300.0, 'DRONE CAP': 2.27, 'TRUCK SPEED': 30.0, 'DRONE SPEED': 40.0, 'DRONE ENDURANCE': 0.6, 'DRONE TIME LIMIT': 1.5, 'TRUCK TIME LIMIT': 1.5, 'TRUCK UNIT COST': 1.25, 'DRONE UNIT COST': 2.03}
Coordinate:
(10.0, 10.0)
(8.08, 0.81)
(10.07, 16.46)
(8.15, 10.68)
(4.47, 3.39)
(13.91, 12.49)
(13.75, 19.07)
(1.87, 10.74)
(1.88, 14.08)
(17.35, 15.64)
(17.21, 16.95)
Pesi:
0.0
1.9
1.02
0.18
0.93
1.04
48.34
1.82
0.98
1.37
1.04


In [24]:
def distance_matrix_computation(coord):
    coord_array = np.array(coord)
    return distance_matrix(coord_array, coord_array)

In [25]:
d_matrix= distance_matrix_computation(coordinates)
t_t_matrix = d_matrix/values["TRUCK SPEED"]
t_d_vector = d_matrix[0]/values["DRONE SPEED"]

In [26]:
model = solver.PDSVRPModel(C_T= values["TRUCK UNIT COST"], 
                    C_D=values["DRONE UNIT COST"],
                    D=values["NUM DRONES"],
                    h=values["NUM TRUCKS"],
                    distances= d_matrix,
                    t_t= t_t_matrix,
                    t_d= t_d_vector,
                    Q_t= values["TRUCK CAP"],
                    Q_d= values["DRONE CAP"],
                    T_t= values["TRUCK TIME LIMIT"],
                    T_d= values["DRONE TIME LIMIT"],
                    w=weights)

In [27]:
model.build_model()
model.solve()
model.print_results()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 146 rows, 270 columns and 1012 nonzeros
Model fingerprint: 0xb6477a59
Variable types: 120 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 161.7715743


Presolve removed 11 rows and 59 columns
Presolve time: 0.00s
Presolved: 135 rows, 211 columns, 916 nonzeros
Variable types: 65 continuous, 146 integer (146 binary)

Root relaxation: objective 4.808852e+01, 67 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   48.08852    0   23  161.77157   48.08852  70.3%     -    0s
H    0     0                     159.3004688   48.08852  69.8%     -    0s
H    0     0                     100.6802114   48.08852  52.2%     -    0s
H    0     0                      99.7538344   48.08852  51.8%     -    0s
     0     0   68.34120    0   13   99.75383   68.34120  31.5%     -    0s
H    0     0                      79.2196480   68.34120  13.7%     -    0s
H    0     0                      74.6407633   68.34120  8.44%     -    0s
     0     0   70.77440    0   20   74.64076   70.77440  5.18%     -   